In [33]:
import os
import numpy as np

if not os.path.basename(os.getcwd()) == "ThousandSuns":
    os.chdir("..")  # Run Once!
%load_ext autoreload
%autoreload 2

from output_graphs.hourly_graph_creator import *
from df_objects.df_objects import ElectricityUseDf, DemandDf, ProductionDf, CostElectricityDf
from hourly_simulation.parameters import simulation_params, Params
from hourly_simulation.predict_demand import predict_demand_in_year
from hourly_simulation.strategies import selling_strategy
from hourly_simulation.parameters import simulation_params, NORMALISED_SOLAR_PRODUCTION
from tests.sanity_checks import test_simulation
import logging

logging.getLogger().setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
num_batteries = 1
print("*Under production*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [9] * 24}))
price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": [100 for i in range(80, 104)],
                                                           "HourOfYear": [i for i in range(80, 104)]}))
binary_price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": [0 for i in range(80, 104)],
                                                           "HourOfYear": [i for i in range(80, 104)]}))

simulation_params_dict = simulation_params._asdict()
simulation_params_dict['MAX_SELLING_POWER'] = 1
simulation_params_dict['CHARGE_POWER'] = 1
simulation_params_dict['BATTERY_CAPACITY'] = 2
simulation_params = Params(**simulation_params_dict)
electricity_use: ElectricityUseDf = selling_strategy.first_selling_strategy(
    demand=future_demand, production=total_panel_production, param= simulation_params, number_of_batteries=num_batteries)
daily_graph(electricity_use.df)
test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
                params=simulation_params, num_batteries=num_batteries)

*Under production*


INFO:root:Passed test_non_negative
INFO:root:Passed test_demand_is_reached
INFO:root:passed test_production_is_used
INFO:root:passed test_all_stored_is_used


AssertionError: -0.05 <= -0.26 <= 2.05

In [40]:
print("*Over production*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [11] * 24}))
electricity_use: ElectricityUseDf = selling_strategy.first_selling_strategy(
    demand=future_demand, production=total_panel_production, param= simulation_params, number_of_batteries=num_batteries)
daily_graph(electricity_use.df)
test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
                params=simulation_params, num_batteries=num_batteries)

*Over production*


INFO:root:Passed test_non_negative
INFO:root:Passed test_demand_is_reached
INFO:root:passed test_production_is_used
INFO:root:passed test_all_stored_is_used
INFO:root:passed test_battery_capacity_is_not_passed
INFO:root:passed test_charge_power_not_passed
INFO:root:passed test_selling_limit_not_passed
INFO:root:Passed all tests


In [41]:
print("*Oscillations around demand*")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [10 + 2 * np.sin(2 * np.pi * x / 6)
                                                                                      for x in range(24)]}))
electricity_use: ElectricityUseDf = selling_strategy.first_selling_strategy(
    demand=future_demand, production=total_panel_production, param= simulation_params, number_of_batteries=num_batteries)
daily_graph(electricity_use.df)
test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
                params=simulation_params, num_batteries=num_batteries)

*Oscillations around demand*


INFO:root:Passed test_non_negative
INFO:root:Passed test_demand_is_reached
INFO:root:passed test_production_is_used
INFO:root:passed test_all_stored_is_used


AssertionError: 2.0 <= 1

In [42]:
num_batteries = 1
print("*Under production*")
binary_cost_list = [0] * 17 + [1] * 7
cost_list = [10] * 17 + [20] * 7

future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [9] * 24}))
price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": cost_list,
                                                           "HourOfYear": [i for i in range(80, 104)]}))
binary_price: CostElectricityDf = CostElectricityDf(pd.DataFrame({"Cost ILS/Kwh": binary_cost_list,
                                                           "HourOfYear": [i for i in range(80, 104)]}))

# electricity_use: ElectricityUseDf = strategies.first_selling_strategy(
#     demand=future_demand, production=total_panel_production,
#     battery_capacity=num_batteries * simulation_params.BATTERY_CAPACITY,
#     battery_power=num_batteries * simulation_params.CHARGE_POWER, battery_efficiency=0.87, cost_profile=price, binary_cost_profile=binary_price, sell_profile=price, sale_max_power=0.1)

simulation_params_dict = simulation_params._asdict()
simulation_params_dict['MAX_SELLING_POWER'] = 10
simulation_params_dict['CHARGE_POWER'] = 4
simulation_params_dict['BATTERY_CAPACITY'] = 2
simulation_params = Params(**simulation_params_dict)
print("*Oscillating production")
future_demand: DemandDf = DemandDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                 "Demand": [10] * 24}))
total_panel_production: ProductionDf = ProductionDf(pd.DataFrame({"HourOfYear": [i for i in range(80, 104)],
                                                                  "SolarProduction": [10 + 2 * np.sin(2 * np.pi * x / 6)
                                                                                      for x in range(24)]}))
electricity_use: ElectricityUseDf = selling_strategy.first_selling_strategy(
    demand=future_demand, production=total_panel_production, param= simulation_params, number_of_batteries=num_batteries)
daily_graph(electricity_use.df)
test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
                params=simulation_params, num_batteries=num_batteries)

*Under production*
*Oscillating production


INFO:root:Passed test_non_negative
INFO:root:Passed test_demand_is_reached
INFO:root:passed test_production_is_used
INFO:root:passed test_all_stored_is_used
INFO:root:passed test_battery_capacity_is_not_passed
INFO:root:passed test_charge_power_not_passed
INFO:root:passed test_selling_limit_not_passed
INFO:root:Passed all tests


In [44]:
num_batteries = 2.1
solar_panel_power_kw = 5437
demand = DemandDf(pd.read_csv(r'data/simulation_demand_input/consumption_hatzor.csv', index_col=0))
future_demand = predict_demand_in_year(demand, params=simulation_params, simulated_year=2020)
total_panel_production = ProductionDf(NORMALISED_SOLAR_PRODUCTION.df.copy())
total_panel_production.df[total_panel_production.SolarProduction] *= solar_panel_power_kw
selling_cost = CostElectricityDf(pd.read_csv(r'data/electricity_cost.csv', index_col=0))
binary_cost = CostElectricityDf(pd.read_csv(r'data/electricity_cost_binary.csv', index_col=0))
simulation_params_dict = simulation_params._asdict()
simulation_params_dict['MAX_SELLING_POWER'] = 14000
simulation_params_dict['CHARGE_POWER'] = 2000
simulation_params_dict['BATTERY_CAPACITY'] = 4000
simulation_params = Params(**simulation_params_dict)
electricity_use: ElectricityUseDf = selling_strategy.first_selling_strategy(
    demand=future_demand, production=total_panel_production, param= simulation_params, number_of_batteries=num_batteries)
yearly_graph(electricity_use.df, batteries_num=num_batteries, batteries_cap=num_batteries * simulation_params.BATTERY_CAPACITY)  # todo: problem with yearly graph...
test_simulation(electricity_use=electricity_use, demand=future_demand, production=total_panel_production,
                params=simulation_params, num_batteries=num_batteries)

AttributeError: 'int' object has no attribute 'df'

In [45]:
electricity_use